In [ ]:
import speckle_fcns_v7 as fcns
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import sys
from tkinter import Tk
from tkinter.filedialog import askdirectory

# bring file selection dialog to front
root = Tk()
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

import ipywidgets as widgets
#%matplotlib widget
%matplotlib qt

plt.close('all')

In [ ]:
### specify directory to seach for data (not needed for scanning computers) ###
scannername = 'strickland' #'strickland' #'franklin' #'strickland'
datapath = 'S:/Schrodinger' #'S:/Franklin' #'S:/Strickland'
if sys.platform == 'Darwin':
    datapath = '~/aws/S3/owi-scan-data/Schrodinger' #'S:/Franklin' #'S:/Strickland'
if os.getlogin().lower() == 'lab': 
    scannername = os.environ['COMPUTERNAME'].lower()
    datapath = 'C:/data_scans/' + scannername
scanorder = 'hhc30' #'hhc30' #'generic30' #'zigzag' #'circle'

In [ ]:
#########################################
###### do not modify below ##############
#########################################

In [ ]:
scanname = askdirectory(title='Select Scan', initialdir=datapath)
with open(scanname + '/scan_metadata.json') as json_file:
    json_dict = json.load(json_file)
if not (json_dict.get('cameraParameters').get('calibrationFolder') is None):
    calname = json_dict.get('cameraParameters').get('calibrationFolder')
    if calname == '':
        calname = askdirectory(title='Select Calibration', initialdir=datapath)
    elif scanname[:scanname.rindex('/')] != calname[:calname.rindex('/')]:
        calname = scanname[:scanname.rindex('/')] + calname[calname.rindex('/'):]
    if (not os.path.exists(calname)):
        calname = askdirectory(title='Select Calibration', initialdir=datapath)    
else:
    calname = askdirectory(title='Select Calibration', initialdir=datapath)

In [ ]:
### data and algo parameters ###
data_dict = fcns.getdatadictionary(scanorder, calname, scanname, scannername)
params_dict = fcns.getparamsdictionary(False)
data_dict, params_dict = fcns.getjsoninfo(data_dict, params_dict)
data_dict['scanorder'] = scanorder

In [ ]:
### load calibration data ###
caldata = fcns.read_bloodflow_protocsv2(data_dict.get('calname') + '/data.csv')
caldata = fcns.sortbycamera_proto2(caldata, data_dict['cids'], 2*data_dict['nt'])
caldata = fcns.removedark2(caldata, data_dict)
caldata = fcns.averagetimeseries_proto(caldata)
params_dict['I0'] = caldata['mean_mean']

In [ ]:
### read log file ###
npositions = data_dict.get('npositions', 30)
completedscan, nattempts = fcns.readlogfile(data_dict.get('dataname') + '/log.txt', npositions)

In [ ]:
### load data ###
data = [{}] * npositions    
for i in range(npositions): 
    data[i] = fcns.read_bloodflow_protocsv2(data_dict.get('dataname') + '/location_' + str(i+1) + '.csv')
    data[i] = fcns.sortbycamera_proto2(data[i], data_dict['cids'], 2*data_dict['nt'])
    data[i] = fcns.removedark2(data[i], data_dict)
    data[i] = fcns.shotnoise(data[i], data_dict)
    data_dict['nstart']=2  #crop data because 1st pt. different and
    data_dict['nstop']=-2  #Octopus sometimes leaves out final data point
    if ('nstart' in data_dict.keys()) or ('nstop' in data_dict.keys()):
        data[i] = fcns.croptime_proto(data[i], data_dict)
    #data[i] = fcns.getpulse(data[i], data_dict)
    data[i]['completedscan'] = completedscan[i]
    data[i]['nattempts'] = np.array([nattempts[i]])

In [ ]:
### get rid of offsets ###
data_dict['offset'] = 0 # 0: none, 1: ax+b, 2: svd with 1/x term
data = fcns.removeOffsetAll(data, data_dict)

In [ ]:
### pulse
for i in range(npositions):
    data[i] = fcns.getpulse(data[i], data_dict)
    data[i] = fcns.highpass(data[i], data_dict)
    data[i] = fcns.goldenpulse(data[i], data_dict, 'contrast')
    data[i] = fcns.auc(data[i], -1, 'contrast')

In [ ]:
### average data ###
for i in range(npositions):
    data[i]=fcns.averagetimeseries_proto(data[i])
    data[i]=fcns.getridofzeros(data[i])
    data[i] = fcns.amplitude(data[i], 'contrast')

In [ ]:
### plot data before calibration ###

# works best with '%matplotlib widget' set at top of notebook (displayedinteractive plots inline)
fcns.plotscan_slider(data, data_dict, 'contrast')

# Used for 'generic30' scanorder 
data_dict['groupNameList'] = ['Group1', 'Group2', 'Group3', 'Group4', 'Group5']
# data_dict['legendNameList'] = ['Camera1', 'Camera2']

#fcns.showAverageData(data, data_dict, 'pulse')
#fcns.plotscandataintime(data, data_dict)
#fcns.plotscandata(data, data_dict, 'mean')
#fcns.plotscandata(data, data_dict, 'std')
#fcns.plotscandata(data, data_dict, 'pcontrast')
fcns.plotscandata(data, data_dict, 'contrast')
#fcns.showAverageData(data, data_dict, 'mean_mean')
fcns.showAverageData(data, data_dict, 'contrast_mean')
#fcns.showResult(data, data_dict, 'nattempts')
fcns.showAverageData(data, data_dict, 'modulationdepth')
fcns.showAverageData(data, data_dict, 'amplitude')
fcns.plottogether(data, data_dict, 'contrast_waveform', -1)
fcns.showAverageData(data, data_dict, 'contrast_auc')
fcns.showAverageData(data, data_dict, 'contrast_waveform_AmpOverStd_mean')

In [ ]:
################################
##### fit for parameter(s) #####
################################

In [ ]:
### attenuation ###

### define independent variables
c2u = params_dict.get('cameras2use')        
p = np.squeeze(params_dict['rho'])[c2u]
pp = np.linspace(np.amin(p), np.amax(p))
pp2 = np.linspace(0, np.amax(p))
exp = params_dict['exp']
expp = np.array([params_dict['exp'][0]])

### calibrate intensity ###
ci = fcns.getIntensityCalibration(params_dict)
for i in range(npositions):
    data[i]['mean'] *= ci
    data[i]['mean_mean'] *= ci
    data[i]['std'] *= ci
    data[i]['std_mean'] *= ci

### find mu_eff & mua ###
mean = np.zeros((npositions, len(c2u), 1))
std = np.zeros((npositions, len(c2u), 1))
for i in range(npositions):
    mean[i, :, 0]  = data[i]['mean_mean'][c2u]
    std[i, :, 0] = data[i]['std_mean'][c2u]
mu_eff_mean, mua_mean   = fcns.fitattenuation(mean,  p, pp, params_dict['musp'], std, params_dict['displayfits'])
for i in range(npositions):
    data[i]['mu_eff_mean'] = mu_eff_mean[i]
    data[i]['mua_mean'] = mua_mean[i]

In [ ]:
### flow ###

### define independent variables
params_dict['C0']=data[0]['contrast_mean'] # relative BFI only
params_dict['mua']=data[0]['mua_mean']
#params_dict['cameras2use'] = np.array([0, 1, 2, 3]) #punting on 35 mm camera for now
c2u = params_dict.get('cameras2use')
p = np.squeeze(params_dict['rho'])[c2u]
pp = np.linspace(np.amin(p), np.amax(p))
pp2 = np.linspace(0, np.amax(p))

### calibrate contrast ###
cc = fcns.getContrastCalibration(params_dict) 
for i in range(npositions):   
    data[i]['contrast'] *= cc
    data[i]['contrast_mean'] *= cc

### find alpha*Db ###
contrast = np.zeros((npositions, len(c2u), 1))
for i in range(npositions):
    contrast[i, :, 0] = data[i]['contrast_mean'][c2u]
alphaDb_mean, beta, offset    = fcns.fitflow_multi(params_dict, contrast,  p, pp2, exp, expp, mua_mean)  #using 1st mua
for i in range(npositions):
    data[i]['rCBF_mean'] = alphaDb_mean[i]

In [ ]:
### plot fitted parameters ###
fcns.showResult(data, data_dict, 'mua_mean')
fcns.showResult(data, data_dict, 'rCBF_mean')